<a href="https://colab.research.google.com/github/ConstructoDestructo/Diabetes_AI_Instrument/blob/main/CDC_NHANES_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
!pip install kneed
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from kneed import KneeLocator
import numpy as np
import matplotlib.pyplot as plt

# --- CONFIG ---
BASE_CYCLE_INDEX = "https://wwwn.cdc.gov/nchs/nhanes/continuousnhanes/default.aspx"
COMPONENTS = ["Demographics", "Dietary", "Examination", "Laboratory", "Questionnaire"]
OUTPUT_JSON = "nhanes_catalog.json"
OUTPUT_CSV = "nhanes_catalog.csv"
HEADERS = {"User-Agent": "Mozilla/5.0 (Data Research Bot; +https://example.com)"}
EXCLUDE = ["What We Eat in America", "Notice to Users"]

# --- Helper functions ---
def get_available_cycles():
    res = requests.get(BASE_CYCLE_INDEX, headers=HEADERS)
    soup = BeautifulSoup(res.text, "html.parser")
    cycles = []
    for link in soup.find_all("a", href=True):
        href = link["href"]
        link_text = link.text.strip()
        if ("BeginYear=" in href or "Cycle=" in href) and "NHANES" in link_text:
            if any(exclude in link_text for exclude in EXCLUDE):
                continue
            if link_text not in cycles:
                cycles.append(link_text)
    return cycles

def get_component_datasets(cycle, component):
    url = f"https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?Component={component}&Cycle={cycle}"
    res = requests.get(url, headers=HEADERS)
    if res.status_code != 200:
        print(f"⚠️ Failed to fetch {component} data for {cycle} (status {res.status_code})")
        return []

    soup = BeautifulSoup(res.text, "html.parser")
    table = soup.find("table", {"id": "GridView1"})
    if not table:
        return []

    cycle_years_match = re.findall(r'\d{4}', cycle)
    if len(cycle_years_match) == 2:
        cycle_start_year = int(cycle_years_match[0])
        cycle_end_year = int(cycle_years_match[1])
    elif len(cycle_years_match) == 1:
        cycle_start_year = cycle_end_year = int(cycle_years_match[0])
    else:
        cycle_start_year = cycle_end_year = None

    records = []
    rows = table.find_all("tr")[1:]  # skip header
    for row in rows:
        cols = row.find_all("td")
        if len(cols) < 4:
            continue
        dataset_name = cols[0].text.strip()
        years = cols[1].text.strip()
        data_link = cols[2].find("a")["href"] if cols[2].find("a") else None
        doc_link = cols[3].find("a")["href"] if cols[3].find("a") else None

        if re.match(r'\d{4}-\d{4}', dataset_name):
            dataset_name, years = years, dataset_name

        if data_link and data_link.startswith("/"):
            data_link = "https://wwwn.cdc.gov" + data_link
        if doc_link and doc_link.startswith("/"):
            doc_link = "https://wwwn.cdc.gov" + doc_link

        dataset_years_match = re.search(r'(\d{4})-(\d{4})', years)
        if dataset_years_match:
            dataset_start_year = int(dataset_years_match.group(1))
            dataset_end_year = int(dataset_years_match.group(2))
            if cycle_start_year and cycle_end_year:
                if dataset_start_year < cycle_start_year or dataset_end_year > cycle_end_year:
                    continue

        print(f"   [Found] {dataset_name} ({years})")
        records.append({
            "cycle": cycle,
            "component": component,
            "dataset_name": dataset_name,
            "years": years,
            "data_url": doc_link,  # swapped on purpose
            "doc_url": data_link
        })
    return records

# --- Crawl NHANES ---
def crawl_nhanes():
    all_records = []
    cycles = get_available_cycles()
    print(f"✅ Found {len(cycles)} valid NHANES cycles: {cycles}")

    for cycle in tqdm(cycles, desc="Crawling NHANES cycles"):
        print(f"\n🔹 Scanning cycle: {cycle}")

        # Always include Demographics first
        try:
            demographics_datasets = get_component_datasets(cycle, "Demographics")
            if demographics_datasets:
                all_records.extend(demographics_datasets)
                print(f"   [Added] Demographics dataset for {cycle}")
            time.sleep(1)
        except Exception as e:
            print(f"⚠️ Error fetching demographics for {cycle}: {e}")

        # Add the rest
        for component in COMPONENTS:
            if component == "Demographics":
                continue
            try:
                datasets = get_component_datasets(cycle, component)
                all_records.extend(datasets)
                time.sleep(1)
            except Exception as e:
                print(f"⚠️ Error on {cycle} - {component}: {e}")

    df = pd.DataFrame(all_records)
    df.to_json(OUTPUT_JSON, orient="records", indent=2)
    df.to_csv(OUTPUT_CSV, index=False)
    print(f"\n✅ Crawling complete! Saved {len(df)} records to {OUTPUT_JSON} and {OUTPUT_CSV}.")
    return df

# --- Run crawler ---
if __name__ == "__main__":
    df_catalog = crawl_nhanes()

   # --- Separate demographics first ---
df_demographics = df_catalog[df_catalog["component"] == "Demographics"].copy()
df_non_demographics = df_catalog[df_catalog["component"] != "Demographics"].copy()

print(f"🔹 Demographics datasets preserved: {len(df_demographics)}")
print(f"🔹 Non-demographics datasets to filter: {len(df_non_demographics)}")

# --- Step 1: AI Filter: Biomedical Expansion on non-demographics datasets ---
SEED_KEYWORDS = [
    "diabetes", "prediabetes", "glucose", "fasting glucose", "hba1c",
    "hemoglobin a1c", "insulin", "c-peptide", "triglyceride", "cholesterol",
    "hdl", "ldl", "body mass index", "bmi", "waist", "weight", "obesity",
    "hypertension"
]

BIOMED_SYNONYMS = {
    "diabetes": ["type 2 diabetes", "type 1 diabetes", "hyperglycemia", "impaired glucose tolerance", "diabetic", "glucose intolerance"],
    "prediabetes": ["impaired fasting glucose", "impaired glucose tolerance", "borderline diabetes"],
    "glucose": ["blood sugar", "serum glucose", "fasting glucose", "plasma glucose"],
    "hba1c": ["hemoglobin a1c", "glycated hemoglobin", "glycohemoglobin", "a1c"],
    "insulin": ["fasting insulin", "serum insulin", "insulin resistance", "c-peptide"],
    "cholesterol": ["hdl", "ldl", "triglyceride", "lipids", "total cholesterol"],
    "obesity": ["overweight", "adiposity", "body mass index", "bmi", "waist circumference"],
    "hypertension": ["high blood pressure", "systolic", "diastolic"],
    "bmi": ["body mass index", "obesity indicator"],
    "triglyceride": ["blood lipids", "fatty acids", "serum triglycerides"],
    "hdl": ["good cholesterol"],
    "ldl": ["bad cholesterol"],
}

def expand_biomedical_keywords(seed_keywords, synonym_map):
    expanded = set(seed_keywords)
    for kw in seed_keywords:
        if kw.lower() in synonym_map:
            expanded.update(synonym_map[kw.lower()])
    return list(expanded)

expanded_keywords = expand_biomedical_keywords(SEED_KEYWORDS, BIOMED_SYNONYMS)
print(f"✅ Biomedical expansion complete: {len(expanded_keywords)} keywords")

NOISE_WORDS = set([
    "exam", "second", "file", "test", "data", "sheet", "survey",
    "questionnaire", "study", "participant", "demographics", "sample",
    "component"
])
dataset_texts = (df_non_demographics["dataset_name"] + " " + df_non_demographics["years"]).tolist()

# --- First-pass TF-IDF ---
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer(ngram_range=(1,1))
tfidf_matrix = vectorizer.fit_transform(dataset_texts + [" ".join(expanded_keywords)])
seed_vector = tfidf_matrix[-1].toarray().flatten()
vocab = vectorizer.get_feature_names_out()
top_indices = seed_vector.argsort()[-50:][::-1]
first_pass_candidates = [vocab[i] for i in top_indices if vocab[i].lower() not in NOISE_WORDS]
first_pass_candidates = list(dict.fromkeys(first_pass_candidates))

# --- Second-pass TF-IDF ---
second_pass_doc = " ".join(expanded_keywords + first_pass_candidates)
vectorizer2 = TfidfVectorizer(ngram_range=(1,1))
tfidf_matrix2 = vectorizer2.fit_transform(dataset_texts + [second_pass_doc])
second_vector = tfidf_matrix2[-1].toarray().flatten()
vocab2 = vectorizer2.get_feature_names_out()
top_indices2 = second_vector.argsort()[-50:][::-1]
second_pass_candidates = [vocab2[i] for i in top_indices2 if vocab2[i].lower() not in NOISE_WORDS]
second_pass_candidates = list(dict.fromkeys(second_pass_candidates))

# --- Combine candidates with expanded keywords ---
final_keywords = list(dict.fromkeys(expanded_keywords + first_pass_candidates + second_pass_candidates))
keyword_doc = " ".join(expanded_keywords * 3 + first_pass_candidates + second_pass_candidates)
vectorizer_final = TfidfVectorizer(ngram_range=(1,1))
tfidf_final = vectorizer_final.fit_transform(dataset_texts + [keyword_doc])
dataset_matrix_final = tfidf_final[:-1]
keyword_vector_final = tfidf_final[-1]
similarities = cosine_similarity(dataset_matrix_final, keyword_vector_final)
df_non_demographics["relevance_score"] = similarities.flatten()

# --- Filter datasets using a threshold ---
threshold = 0.05
df_ai_filtered = df_non_demographics[df_non_demographics["relevance_score"] >= threshold].sort_values(by="relevance_score", ascending=False)
print(f"\n✅ AI filter applied: {len(df_ai_filtered)} datasets selected (excluding demographics)")

# --- Step 2: Semantic Refinement ---
from sentence_transformers import SentenceTransformer
from kneed import KneeLocator
import matplotlib.pyplot as plt
import numpy as np

apply_semantic_refinement = True
min_datasets = 145
plot_threshold = True

if apply_semantic_refinement:
    df_step2 = df_ai_filtered.copy()

    print("🧠 Computing sentence embeddings...")
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    dataset_texts_step2 = (df_step2["dataset_name"] + " " + df_step2["years"]).tolist()
    embeddings = model.encode(dataset_texts_step2, batch_size=32, show_progress_bar=True)
    keyword_embedding = model.encode(" ".join(final_keywords))

    cosine_similarities = np.dot(embeddings, keyword_embedding) / (
        np.linalg.norm(embeddings, axis=1) * np.linalg.norm(keyword_embedding)
    )
    df_step2["semantic_score"] = cosine_similarities

    # Dynamic threshold via KneeLocator
    scores_sorted = np.sort(df_step2["semantic_score"].values)[::-1]
    x = np.arange(len(scores_sorted))
    y = scores_sorted
    knee = KneeLocator(x, y, curve='convex', direction='decreasing')

    if knee.knee is not None:
        dynamic_threshold = y[knee.knee]
    else:
        dynamic_threshold = 0.0

    df_dynamic_filtered = df_step2[df_step2["semantic_score"] >= dynamic_threshold]

    if len(df_dynamic_filtered) < min_datasets:
        df_dynamic_filtered = df_step2.sort_values(by="semantic_score", ascending=False).iloc[:min_datasets]

    df_dynamic_filtered = df_dynamic_filtered.sort_values(by="semantic_score", ascending=False)

# --- Merge demographics back ---
df_final = pd.concat([df_dynamic_filtered, df_demographics], ignore_index=True)
df_final.to_csv("nhanes_ai_semantic_dynamic.csv", index=False)
print(f"✅ Dynamic semantic refinement complete: {len(df_final)} datasets saved → nhanes_ai_semantic_dynamic.csv")





















✅ Found 13 valid NHANES cycles: ['NHANES 08/2021-08/2023', 'NHANES 2017-March 2020', 'NHANES 2019-2020', 'NHANES 2017-2018', 'NHANES 2015-2016', 'NHANES 2013-2014', 'NHANES 2011-2012', 'NHANES 2009-2010', 'NHANES 2007-2008', 'NHANES 2005-2006', 'NHANES 2003-2004', 'NHANES 2001-2002', 'NHANES 1999-2000']


Crawling NHANES cycles:   0%|          | 0/13 [00:00<?, ?it/s]


🔹 Scanning cycle: NHANES 08/2021-08/2023
   [Found] Demographic Variables and Sample Weights (2021-2023)
   [Added] Demographics dataset for NHANES 08/2021-08/2023
   [Found] Dietary Interview - Individual Foods, First Day (2021-2023)
   [Found] Dietary Interview - Individual Foods, Second Day (2021-2023)
   [Found] Dietary Interview - Total Nutrient Intakes, First Day (2021-2023)
   [Found] Dietary Interview - Total Nutrient Intakes, Second Day (2021-2023)
   [Found] Dietary Interview Technical Support File - Food Codes (2021-2023)
   [Found] Dietary Supplement Use 30-Day - Individual Dietary Supplements (2021-2023)
   [Found] Dietary Supplement Use 30-Day - Total Dietary Supplements (2021-2023)
   [Found] Balance (2021-2023)
   [Found] Blood Pressure - Oscillometric Measurements (2021-2023)
   [Found] Body Measures (2021-2023)
   [Found] Liver Ultrasound Transient Elastography (2021-2023)
   [Found] Albumin & Creatinine - Urine (2021-2023)
   [Found] alpha-1-Acid Glycoprotein (2021-

Crawling NHANES cycles:   8%|▊         | 1/13 [00:09<01:53,  9.42s/it]


🔹 Scanning cycle: NHANES 2017-March 2020
   [Found] Demographic Variables and Sample Weights (2017-2018)
   [Found] Demographic Variables and Sample Weights (2017-2020)
   [Added] Demographics dataset for NHANES 2017-March 2020
   [Found] Dietary Interview - Individual Foods, First Day (2017-2018)
   [Found] Dietary Interview - Individual Foods, First Day (2017-2020)
   [Found] Dietary Interview - Individual Foods, Second Day (2017-2018)
   [Found] Dietary Interview - Individual Foods, Second Day (2017-2020)
   [Found] Dietary Interview - Total Nutrient Intakes, First Day (2017-2018)
   [Found] Dietary Interview - Total Nutrient Intakes, First Day (2017-2020)
   [Found] Dietary Interview - Total Nutrient Intakes, Second Day (2017-2018)
   [Found] Dietary Interview - Total Nutrient Intakes, Second Day (2017-2020)
   [Found] Dietary Interview Technical Support File - Food Codes (2017-2018)
   [Found] Dietary Interview Technical Support File - Food Codes (2017-2020)
   [Found] Dietary Su

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

✅ Dynamic semantic refinement complete: 158 datasets saved → nhanes_ai_semantic_dynamic.csv


In [2]:
import os
!pip install pyreadstat
import requests
import pyreadstat
import pandas as pd
from tqdm import tqdm
import time

OUTPUT_DIR = "nhanes_data"
os.makedirs(OUTPUT_DIR, exist_ok=True)

df_filtered = pd.read_csv("nhanes_ai_semantic_dynamic.csv")

MAX_RETRIES = 5
RETRY_DELAY = 2

for idx, row in tqdm(df_filtered.iterrows(), total=len(df_filtered), desc="Downloading NHANES datasets"):
    dataset_name = row["dataset_name"]
    xpt_url = row["data_url"]

    if not isinstance(xpt_url, str) or not xpt_url.strip():
        print(f"⚠️ Skipping {dataset_name}: No URL provided")
        continue

    # Clean URL thoroughly
    xpt_url = xpt_url.strip().replace("\n", "").replace("\r", "")
    if xpt_url.startswith("/"):
        xpt_url = "https://wwwn.cdc.gov" + xpt_url

    # Use cycle/component in filename to prevent collisions
    cycle = row.get("cycle", "UnknownCycle")
    component = row.get("component", "UnknownComponent")
    filename = f"{cycle}_{component}_{dataset_name}".replace("/", "_").replace(" ", "_") + ".csv"
    filepath = os.path.join(OUTPUT_DIR, filename)

    if os.path.exists(filepath):
        continue

    success = False
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            resp = requests.get(xpt_url, timeout=30, allow_redirects=True)
            resp.raise_for_status()

            # Save temp .XPT
            with open("temp.xpt", "wb") as f:
                f.write(resp.content)

                # Convert to CSV
                df_xpt, meta = pyreadstat.read_xport("temp.xpt", encoding="latin1")

            # Add metadata
            df_xpt["NHANES_Cycle"] = cycle
            df_xpt["Component"] = component
            df_xpt["Dataset_Name"] = dataset_name
            df_xpt["Doc_URL"] = row.get("doc_url", "")

            df_xpt.to_csv(filepath, index=False)
            os.remove("temp.xpt")

            print(f"✅ Saved {filename}")
            success = True
            time.sleep(0.2)
            break

        except Exception as e:
            print(f"⚠️ Attempt {attempt} failed for {dataset_name}: {e}")
            time.sleep(RETRY_DELAY)

    if not success:
        print(f"❌ Failed to download {dataset_name}")

print(f"\n✅ Download complete. Files saved in '{OUTPUT_DIR}'")








   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 666.4/666.4 kB 7.7 MB/s eta 0:00:00


✅ Saved NHANES_2001-2002_Laboratory_Glycohemoglobin,_Plasma_Glucose,_Serum_C-peptide,_&_Insulin,_Second_Exam.csv


✅ Saved NHANES_2001-2002_Laboratory_Plasma_Fasting_Glucose,_Serum_C-peptide_&_Insulin.csv


✅ Saved NHANES_2003-2004_Laboratory_Plasma_Fasting_Glucose,_Serum_C-peptide_&_Insulin.csv


✅ Saved NHANES_1999-2000_Laboratory_Plasma_Fasting_Glucose,_Serum_C-peptide_&_Insulin.csv


...

✅ Saved NHANES_2005-2006_Demographics_Demographic_Variables_&_Sample_Weights.csv


✅ Saved NHANES_2003-2004_Demographics_Demographic_Variables_&_Sample_Weights.csv


✅ Saved NHANES_2001-2002_Demographics_Demographic_Variables_&_Sample_Weights.csv


✅ Saved NHANES_1999-2000_Demographics_Demographic_Variables_&_Sample_Weights.csv



✅ Download complete. Files saved in 'nhanes_data'


In [3]:
import pandas as pd
import os
import re
from tqdm import tqdm

DATA_DIR = "nhanes_data"
OUTPUT_FILE = "nhanes_patient_flattened.csv"
SUMMARY_FILE = "nhanes_column_summary.csv"

# Columns to drop before merging (metadata)
METADATA_COLS = ['Dataset_Name', 'Doc_URL', 'Component', 'NHANES_Cycle']

# Track files missing SEQN
missing_seqn_files = []

# Function to extract NHANES cycle from filename
def extract_cycle(filename):
    match = re.search(r'(\d{4})[-_](\d{4})', filename)
    if match:
        return f"{match.group(1)}-{match.group(2)}"
    numbers = re.findall(r'\d{4}', filename)
    if len(numbers) >= 2:
        return f"{numbers[0]}-{numbers[1]}"
    return "unknown"

# 1️⃣ Group datasets by NHANES cycle
cycle_files = {}
for file in os.listdir(DATA_DIR):
    if file.endswith(".csv"):
        cycle = extract_cycle(file)
        cycle_files.setdefault(cycle, []).append(file)

# 2️⃣ Process each cycle
flattened_dfs = []
missing_datasets = {}
column_sources = []  # Track each column's origin

for cycle, files in tqdm(cycle_files.items(), desc="Processing NHANES cycles"):
    missing_datasets[cycle] = []

    # Find demographics file
    demog_file = [f for f in files if "Demographics" in f]
    if not demog_file:
        print(f"⚠️ No demographics file found for cycle {cycle}. Skipping cycle.")
        missing_datasets[cycle].append("Demographics")
        continue

    # Read demographics
    df_cycle = pd.read_csv(os.path.join(DATA_DIR, demog_file[0]), low_memory=False)
    print(f"✅ Demographics loaded for {cycle}: {len(df_cycle)} patients, {len(df_cycle.columns)} columns")

    # Track column origins for demographics
    for col in df_cycle.columns:
        column_sources.append({
            "Column": col,
            "Cycle": cycle,
            "Source_File": demog_file[0]
        })

    # Merge all other datasets
    for f in files:
        if f == demog_file[0]:
            continue

        df_other = pd.read_csv(os.path.join(DATA_DIR, f), low_memory=False)

        if 'SEQN' not in df_other.columns:
            print(f"⚠️ {f} has no SEQN. Skipping merge.")
            missing_datasets[cycle].append(f)
            missing_seqn_files.append(f"{cycle}: {f}")
            continue

        # Drop metadata columns
        df_other_clean = df_other.drop(columns=[c for c in METADATA_COLS if c in df_other.columns])

        # Rename columns to avoid collisions
        rename_map = {col: f"{col}_{os.path.splitext(f)[0]}" for col in df_other_clean.columns if col != 'SEQN'}
        df_other_clean = df_other_clean.rename(columns=rename_map)

        # Track column origins
        for col in df_other_clean.columns:
            if col != 'SEQN':
                column_sources.append({
                    "Column": col,
                    "Cycle": cycle,
                    "Source_File": f
                })

        try:
            df_cycle = df_cycle.merge(df_other_clean, on="SEQN", how="left")
            print(f"🔹 Merged {f} into {cycle}: +{len(rename_map)} columns")
        except Exception as e:
            print(f"⚠️ Failed to merge {f} into {cycle}: {e}")
            missing_datasets[cycle].append(f)

    flattened_dfs.append(df_cycle)

# 3️⃣ Combine all cycles
all_patients_df = pd.concat(flattened_dfs, ignore_index=True, sort=False)

# 4️⃣ Save flattened dataset
all_patients_df.to_csv(OUTPUT_FILE, index=False)
print(f"\n✅ Flattened dataset ready: {OUTPUT_FILE}")
print(f"Shape: {all_patients_df.shape[0]} patients, {all_patients_df.shape[1]} variables")

# 5️⃣ Save column summary
df_summary = pd.DataFrame(column_sources)
df_summary.to_csv(SUMMARY_FILE, index=False)
print(f"📊 Column summary saved: {SUMMARY_FILE}")

# 6️⃣ Report missing datasets
print("\n📋 Missing datasets per cycle:")
for cycle, missing in missing_datasets.items():
    if missing:
        print(f"- {cycle}: {missing}")

# 7️⃣ Report files skipped due to missing SEQN
if missing_seqn_files:
    print("\n⚠️ Files skipped due to missing SEQN:")
    for f in missing_seqn_files:
        print(f"  - {f}")







Processing NHANES cycles:   0%|          | 0/12 [00:00<?, ?it/s]

✅ Demographics loaded for 2017-2020: 9254 patients, 50 columns
🔹 Merged NHANES_2017-March_2020_Examination_Blood_Pressure_-_Oscillometric_Measurements.csv into 2017-2020: +12 columns
🔹 Merged NHANES_2017-March_2020_Laboratory_Cholesterol_-_Low-Density_Lipoproteins_(LDL)_&_Triglycerides.csv into 2017-2020: +9 columns
🔹 Merged NHANES_2017-March_2020_Laboratory_Fasting_Questionnaire.csv into 2017-2020: +18 columns
🔹 Merged NHANES_2017-March_2020_Laboratory_Plasma_Fasting_Glucose.csv into 2017-2020: +3 columns
🔹 Merged NHANES_2017-March_2020_Examination_Blood_Pressure_-_Oscillometric_Measurement.csv into 2017-2020: +11 columns
🔹 Merged NHANES_2017-March_2020_Questionnaire_Weight_History.csv into 2017-2020: +36 columns
🔹 Merged NHANES_2017-March_2020_Questionnaire_Blood_Pressure_&_Cholesterol.csv into 2017-2020: +10 columns
🔹 Merged NHANES_2017-March_2020_Laboratory_Cholesterol_-_High_-_Density_Lipoprotein_(HDL).csv into 2017-2020: +2 columns
🔹 Merged NHANES_2017-March_2020_Examination_Bloo

Processing NHANES cycles:   8%|▊         | 1/12 [00:00<00:05,  1.92it/s]

🔹 Merged NHANES_2017-March_2020_Laboratory_Folate_Forms_-_Total_&_Individual_-_Serum.csv into 2017-2020: +15 columns
🔹 Merged NHANES_2017-March_2020_Laboratory_Cholesterol_-_Total.csv into 2017-2020: +2 columns
✅ Demographics loaded for 2017-2018: 9254 patients, 50 columns
🔹 Merged NHANES_2017-2018_Questionnaire_Diabetes.csv into 2017-2018: +53 columns
🔹 Merged NHANES_2017-2018_Laboratory_Plasma_Fasting_Glucose.csv into 2017-2018: +3 columns
🔹 Merged NHANES_2017-2018_Laboratory_Fasting_Questionnaire.csv into 2017-2018: +18 columns
🔹 Merged NHANES_2017-2018_Examination_Blood_Pressure_-_Oscillometric_Measurements.csv into 2017-2018: +12 columns
🔹 Merged NHANES_2017-2018_Laboratory_Cholesterol_-_Low-Density_Lipoproteins_(LDL)_&_Triglycerides.csv into 2017-2018: +9 columns
🔹 Merged NHANES_2017-2018_Laboratory_Insulin.csv into 2017-2018: +4 columns
🔹 Merged NHANES_2017-2018_Laboratory_Cholesterol_-_Total.csv into 2017-2018: +2 columns
🔹 Merged NHANES_2017-2018_Laboratory_Cholesterol_-_High_

Processing NHANES cycles:  17%|█▋        | 2/12 [00:00<00:04,  2.08it/s]

...

Processing NHANES cycles:  92%|█████████▏| 11/12 [00:06<00:00,  1.43it/s]

🔹 Merged NHANES_2009-2010_Examination_Blood_Pressure.csv into 2009-2010: +26 columns
🔹 Merged NHANES_2009-2010_Laboratory_Plasma_Fasting_Glucose_&_Insulin.csv into 2009-2010: +7 columns
🔹 Merged NHANES_2009-2010_Laboratory_Cholesterol_-_LDL_&_Triglycerides.csv into 2009-2010: +5 columns
🔹 Merged NHANES_2009-2010_Laboratory_Fasting_Questionnaire.csv into 2009-2010: +18 columns
✅ Demographics loaded for 2007-2008: 10149 patients, 47 columns
🔹 Merged NHANES_2007-2008_Questionnaire_Diabetes.csv into 2007-2008: +35 columns
🔹 Merged NHANES_2007-2008_Laboratory_Cholesterol_-_Total.csv into 2007-2008: +2 columns
🔹 Merged NHANES_2007-2008_Laboratory_Cholesterol_-_LDL_&_Triglycerides.csv into 2007-2008: +5 columns
🔹 Merged NHANES_2007-2008_Laboratory_Glycohemoglobin.csv into 2007-2008: +1 columns
🔹 Merged NHANES_2007-2008_Questionnaire_Blood_Pressure_&_Cholesterol.csv into 2007-2008: +18 columns
🔹 Merged NHANES_2007-2008_Laboratory_Plasma_Fasting_Glucose_&_Insulin.csv into 2007-2008: +7 columns


Processing NHANES cycles: 100%|██████████| 12/12 [00:07<00:00,  1.66it/s]

🔹 Merged NHANES_2007-2008_Laboratory_Fasting_Questionnaire.csv into 2007-2008: +18 columns



✅ Flattened dataset ready: nhanes_patient_flattened.csv
Shape: 122503 patients, 1983 variables
📊 Column summary saved: nhanes_column_summary.csv

📋 Missing datasets per cycle:


In [4]:
!pip install duckdb psutil
import duckdb
import pandas as pd
import time
import os
import re
from datetime import datetime

# ========== CONFIGURATION ==========
INPUT_FILE = "nhanes_patient_flattened.csv"
OUTPUT_FILE = "nhanes_wide_aggregated.csv"

# ========== SETUP ==========
print("=" * 60)
print("🚀 NHANES Wide Format Aggregation (One Row Per Patient)")
print("=" * 60)
print(f"Input:  {INPUT_FILE}")
print(f"Output: {OUTPUT_FILE}")
print(f"Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

start_time = time.time()

# ========== STEP 1: Check Input File ==========
if not os.path.exists(INPUT_FILE):
    raise FileNotFoundError(f"❌ Input file not found: {INPUT_FILE}")

file_size_mb = os.path.getsize(INPUT_FILE) / (1024 ** 2)
print(f"📁 Input file size: {file_size_mb:.2f} MB\n")

# ========== STEP 2: Analyze Column Structure ==========
print("🔍 Analyzing column structure...")
con = duckdb.connect()

# Load first row to understand structure
sample = con.execute(f"""
    SELECT * FROM read_csv('{INPUT_FILE}', AUTO_DETECT=TRUE, SAMPLE_SIZE=1000)
    LIMIT 1
""").df()

print(f"✅ Found {len(sample.columns)} total columns\n")

# ========== STEP 3: Extract Cycle and Base Variable from Column Names ==========
print("🔨 Extracting base variables and cycle information...")

def parse_column_name(col):
    """
    Parse column like: LBXGLU_NHANES_2017-2018_Laboratory_Glucose
    Returns: (base_variable, cycle, full_name)
    """
    if col == 'SEQN':
        return ('SEQN', None, 'SEQN')

    # Try to extract cycle (e.g., 2017-2018)
    cycle_match = re.search(r'(\d{4})[-_](\d{4})', col)
    cycle = f"{cycle_match.group(1)}-{cycle_match.group(2)}" if cycle_match else 'unknown'

    # Base variable is the first part before underscore
    parts = col.split('_')
    base_var = parts[0] if parts else col

    return (base_var, cycle, col)

# Parse all columns
column_info = {}
for col in sample.columns:
    if col == 'SEQN':
        continue
    base_var, cycle, full_name = parse_column_name(col)

    if base_var not in column_info:
        column_info[base_var] = []
    column_info[base_var].append({
        'full_name': full_name,
        'cycle': cycle
    })

print(f"✅ Identified {len(column_info)} unique base variables")
print(f"   Example: {list(column_info.keys())[:5]}\n")

# ========== STEP 4: Build Aggregation Query ==========
print("⚙️  Building aggregation query...")
print("   Strategy: Group by SEQN, concatenate values with cycle info\n")

# Build COALESCE statements for each base variable
# Format: "value1 [cycle1], value2 [cycle2]"
agg_statements = []

for base_var, columns in column_info.items():
    if len(columns) == 1:
        # Only one column for this variable - just select it
        col = columns[0]['full_name']
        agg_statements.append(f'"{col}" AS "{base_var}"')
    else:
        # Multiple columns - concatenate with cycle info
        # Build: CONCAT_WS(', ',
        #          CASE WHEN col1 IS NOT NULL THEN col1 || ' [cycle1]' END,
        #          CASE WHEN col2 IS NOT NULL THEN col2 || ' [cycle2]' END)
        concat_parts = []
        for col_info in columns:
            col = col_info['full_name']
            cycle = col_info['cycle']
            concat_parts.append(
                f'CASE WHEN "{col}" IS NOT NULL THEN CAST("{col}" AS VARCHAR) || \' [{cycle}]\' END'
            )

        concat_expr = f"CONCAT_WS(', ', {', '.join(concat_parts)})"
        agg_statements.append(f'{concat_expr} AS "{base_var}"')

# Build final SELECT statement
select_clause = "SEQN, " + ", ".join(agg_statements)

query = f"""
    COPY (
        SELECT {select_clause}
        FROM read_csv('{INPUT_FILE}',
                     AUTO_DETECT=TRUE,
                     SAMPLE_SIZE=50000,
                     IGNORE_ERRORS=TRUE)
        ORDER BY SEQN
    ) TO '{OUTPUT_FILE}' (HEADER, DELIMITER ',')
"""

# ========== STEP 5: Execute Transformation ==========
print("⚙️  Executing transformation...")
print("    (This may take 10-30 minutes depending on dataset size)")
print("    Processing all patients into single-row format...\n")

try:
    con.execute(query)
    transform_time = time.time() - start_time
    print(f"✅ Transformation complete in {transform_time/60:.2f} minutes\n")

    # ========== STEP 6: Get Statistics ==========
    print("📊 Analyzing results...")

    result_df = pd.read_csv(OUTPUT_FILE, nrows=5)

    print(f"   Output columns: {len(result_df.columns)}")
    print(f"   (One column per unique base variable)\n")

    output_size_mb = os.path.getsize(OUTPUT_FILE) / (1024 ** 2)
    print(f"💾 Output file size: {output_size_mb:.2f} MB\n")

    # ========== STEP 7: Show Sample Data ==========
    print("🔬 Sample of first patient (first 5 columns):")
    print(result_df.iloc[0, :5].to_string())
    print("\n📋 Example of multi-cycle values:")
    # Find a column with commas (multiple values)
    for col in result_df.columns[1:20]:  # Check first 20 columns
        val = result_df.iloc[0][col]
        if isinstance(val, str) and ',' in val:
            print(f"   {col}: {val[:100]}...")
            break
    print()

    # ========== STEP 8: Memory Usage Report ==========
    import psutil
    process = psutil.Process()
    memory_mb = process.memory_info().rss / (1024 ** 2)
    print(f"💾 Peak memory usage: {memory_mb:.2f} MB")

except Exception as e:
    print(f"❌ Error during transformation: {e}")
    import traceback
    traceback.print_exc()
    raise

finally:
    con.close()

# ========== SUMMARY ==========
total_time = time.time() - start_time
print("\n" + "=" * 60)
print("✅ TRANSFORMATION COMPLETE!")
print("=" * 60)
print(f"⏱️  Total time: {total_time/60:.2f} minutes")
print(f"📁 Output file: {OUTPUT_FILE}")
print(f"📊 Format: One row per patient (SEQN)")
print(f"📋 Columns: {len(result_df.columns)} unique variables")
print(f"💡 Multi-cycle values formatted as: value1 [2017-2018], value2 [2019-2020]")
print("=" * 60)

print("\n💡 Excel Tips:")
print("   - This format is Excel-friendly (manageable column count)")
print("   - Each cell contains all values for that patient/variable")
print("   - Cycle info preserved in brackets: [2017-2018]")
print("   - AI can parse: 'value [cycle], value [cycle]' format")






🚀 NHANES Wide Format Aggregation (One Row Per Patient)
Input:  nhanes_patient_flattened.csv
Output: nhanes_wide_aggregated.csv
Started: 2025-10-21 21:34:35

📁 Input file size: 304.39 MB

🔍 Analyzing column structure...
✅ Found 1983 total columns

🔨 Extracting base variables and cycle information...
✅ Identified 607 unique base variables
   Example: ['SDDSRVYR', 'RIDSTATR', 'RIAGENDR', 'RIDAGEYR', 'RIDAGEMN']

⚙️  Building aggregation query...
   Strategy: Group by SEQN, concatenate values with cycle info

⚙️  Executing transformation...
    (This may take 10-30 minutes depending on dataset size)
    Processing all patients into single-row format...



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Transformation complete in 0.62 minutes

📊 Analyzing results...
   Output columns: 608
   (One column per unique base variable)

💾 Output file size: 279.89 MB

🔬 Sample of first patient (first 5 columns):
SEQN        1.0
SDDSRVYR    1.0
RIDSTATR    2.0
RIAGENDR    2.0
RIDAGEYR    2.0

📋 Example of multi-cycle values:

💾 Peak memory usage: 5047.58 MB

✅ TRANSFORMATION COMPLETE!
⏱️  Total time: 0.62 minutes
📁 Output file: nhanes_wide_aggregated.csv
📊 Format: One row per patient (SEQN)
📋 Columns: 608 unique variables
💡 Multi-cycle values formatted as: value1 [2017-2018], value2 [2019-2020]

💡 Excel Tips:
   - This format is Excel-friendly (manageable column count)
   - Each cell contains all values for that patient/variable
   - Cycle info preserved in brackets: [2017-2018]
   - AI can parse: 'value [cycle], value [cycle]' format


In [9]:
"""
CHUNK 8: WIDE FORMAT AGGREGATION
=================================
Transforms flattened patient data into wide format (ONE row per patient)

Input:  nhanes_patient_flattened.csv (from Chunk 7)
Output: nhanes_wide_aggregated.csv

What it does:
- Takes data where patients appear multiple times across different cycles
- Combines into ONE row per patient
- Preserves cycle information like: "120 [2017-2018], 115 [2019-2020]"

Author: Claude
"""

import pandas as pd
import duckdb
import time
import os
import re
from datetime import datetime

# ============================================================================
# CONFIGURATION
# ============================================================================

INPUT_FILE = "nhanes_patient_flattened.csv"
OUTPUT_FILE = "nhanes_wide_aggregated.csv"

# ============================================================================
# MAIN TRANSFORMATION
# ============================================================================

print("=" * 70)
print("🚀 CHUNK 8: WIDE FORMAT AGGREGATION")
print("=" * 70)
print(f"Input:  {INPUT_FILE}")
print(f"Output: {OUTPUT_FILE}")
print(f"Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

start_time = time.time()

# Step 1: Check input file exists
if not os.path.exists(INPUT_FILE):
    raise FileNotFoundError(f"❌ Input file not found: {INPUT_FILE}")

file_size_mb = os.path.getsize(INPUT_FILE) / (1024 ** 2)
print(f"📁 Input file size: {file_size_mb:.2f} MB\n")

# Step 2: Analyze column structure
print("🔍 Analyzing column structure...")
con = duckdb.connect()

sample = con.execute(f"""
    SELECT * FROM read_csv('{INPUT_FILE}', AUTO_DETECT=TRUE, SAMPLE_SIZE=1000)
    LIMIT 1
""").df()

print(f"✅ Found {len(sample.columns)} total columns\n")

# Step 3: Parse column names to extract base variables and cycles
print("🔨 Extracting base variables and cycle information...")

def parse_column_name(col):
    """
    Parse column like: LBXGLU_NHANES_2017-2018_Laboratory_Glucose
    Returns: (base_variable, cycle, full_name)
    """
    if col == 'SEQN':
        return ('SEQN', None, 'SEQN')

    # Extract cycle (e.g., 2017-2018)
    cycle_match = re.search(r'(\d{4})[-_](\d{4})', col)
    cycle = f"{cycle_match.group(1)}-{cycle_match.group(2)}" if cycle_match else 'unknown'

    # Base variable is the first part before underscore
    parts = col.split('_')
    base_var = parts[0] if parts else col

    return (base_var, cycle, col)

# Parse all columns
column_info = {}
for col in sample.columns:
    if col == 'SEQN':
        continue
    base_var, cycle, full_name = parse_column_name(col)

    if base_var not in column_info:
        column_info[base_var] = []
    column_info[base_var].append({
        'full_name': full_name,
        'cycle': cycle
    })

print(f"✅ Identified {len(column_info)} unique base variables")
print(f"   Example: {list(column_info.keys())[:5]}\n")

# Step 4: Build aggregation query
print("⚙️  Building aggregation query...")
print("   Strategy: Group by SEQN, concatenate values with cycle info\n")

agg_statements = []

for base_var, columns in column_info.items():
    if len(columns) == 1:
        # Only one column for this variable - just select it
        col = columns[0]['full_name']
        agg_statements.append(f'"{col}" AS "{base_var}"')
    else:
        # Multiple columns - concatenate with cycle info
        # Format: "value1 [cycle1], value2 [cycle2]"
        concat_parts = []
        for col_info in columns:
            col = col_info['full_name']
            cycle = col_info['cycle']
            concat_parts.append(
                f'CASE WHEN "{col}" IS NOT NULL THEN CAST("{col}" AS VARCHAR) || \' [{cycle}]\' END'
            )

        concat_expr = f"CONCAT_WS(', ', {', '.join(concat_parts)})"
        agg_statements.append(f'{concat_expr} AS "{base_var}"')

# Build final SELECT statement
select_clause = "SEQN, " + ", ".join(agg_statements)

query = f"""
    COPY (
        SELECT {select_clause}
        FROM read_csv('{INPUT_FILE}',
                     AUTO_DETECT=TRUE,
                     SAMPLE_SIZE=50000,
                     IGNORE_ERRORS=TRUE)
        ORDER BY SEQN
    ) TO '{OUTPUT_FILE}' (HEADER, DELIMITER ',')
"""

# Step 5: Execute transformation
print("⚙️  Executing transformation...")
print("    (This may take 10-30 minutes depending on dataset size)")
print("    Processing all patients into single-row format...\n")

try:
    con.execute(query)
    transform_time = time.time() - start_time
    print(f"✅ Transformation complete in {transform_time/60:.2f} minutes\n")

    # Step 6: Get statistics
    print("📊 Analyzing results...")

    result_df = pd.read_csv(OUTPUT_FILE, nrows=5)

    print(f"   Output columns: {len(result_df.columns)}")
    print(f"   (One column per unique base variable)\n")

    output_size_mb = os.path.getsize(OUTPUT_FILE) / (1024 ** 2)
    print(f"💾 Output file size: {output_size_mb:.2f} MB\n")

    # Step 7: Show sample data
    print("🔬 Sample of first patient (first 5 columns):")
    print(result_df.iloc[0, :5].to_string())
    print("\n📋 Example of multi-cycle values:")

    # Find a column with commas (multiple values)
    for col in result_df.columns[1:20]:  # Check first 20 columns
        val = result_df.iloc[0][col]
        if isinstance(val, str) and ',' in val:
            print(f"   {col}: {val[:100]}...")
            break
    print()

    # Step 8: Memory usage report
    import psutil
    process = psutil.Process()
    memory_mb = process.memory_info().rss / (1024 ** 2)
    print(f"💾 Peak memory usage: {memory_mb:.2f} MB")

except Exception as e:
    print(f"❌ Error during transformation: {e}")
    import traceback
    traceback.print_exc()
    raise

finally:
    con.close()

# Summary
total_time = time.time() - start_time
print("\n" + "=" * 70)
print("✅ TRANSFORMATION COMPLETE!")
print("=" * 70)
print(f"⏱️  Total time: {total_time/60:.2f} minutes")
print(f"📁 Output file: {OUTPUT_FILE}")
print(f"📊 Format: One row per patient (SEQN)")
print(f"📋 Columns: {len(result_df.columns)} unique variables")
print(f"💡 Multi-cycle values formatted as: value1 [2017-2018], value2 [2019-2020]")
print("=" * 70)
print()
print("💡 Excel Tips:")
print("   - This format is Excel-friendly (manageable column count)")
print("   - Each cell contains all values for that patient/variable")
print("   - Cycle info preserved in brackets: [2017-2018]")
print("   - AI can parse: 'value [cycle], value [cycle]' format")


🚀 CHUNK 8: WIDE FORMAT AGGREGATION
Input:  nhanes_patient_flattened.csv
Output: nhanes_wide_aggregated.csv
Started: 2025-10-21 21:58:01

📁 Input file size: 304.39 MB

🔍 Analyzing column structure...
✅ Found 1983 total columns

🔨 Extracting base variables and cycle information...
✅ Identified 607 unique base variables
   Example: ['SDDSRVYR', 'RIDSTATR', 'RIAGENDR', 'RIDAGEYR', 'RIDAGEMN']

⚙️  Building aggregation query...
   Strategy: Group by SEQN, concatenate values with cycle info

⚙️  Executing transformation...
    (This may take 10-30 minutes depending on dataset size)
    Processing all patients into single-row format...



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Transformation complete in 0.72 minutes

📊 Analyzing results...
   Output columns: 608
   (One column per unique base variable)

💾 Output file size: 279.89 MB

🔬 Sample of first patient (first 5 columns):
SEQN        1.0
SDDSRVYR    1.0
RIDSTATR    2.0
RIAGENDR    2.0
RIDAGEYR    2.0

📋 Example of multi-cycle values:

💾 Peak memory usage: 4876.62 MB

✅ TRANSFORMATION COMPLETE!
⏱️  Total time: 0.73 minutes
📁 Output file: nhanes_wide_aggregated.csv
📊 Format: One row per patient (SEQN)
📋 Columns: 608 unique variables
💡 Multi-cycle values formatted as: value1 [2017-2018], value2 [2019-2020]

💡 Excel Tips:
   - This format is Excel-friendly (manageable column count)
   - Each cell contains all values for that patient/variable
   - Cycle info preserved in brackets: [2017-2018]
   - AI can parse: 'value [cycle], value [cycle]' format


In [25]:
"""
CHUNK 9: OPTIMIZED DATA DICTIONARY GENERATOR
============================================
MUCH FASTER version using parallel processing and smart caching

Optimizations:
- Parallel URL processing (check multiple URLs at once)
- Early termination (stop checking URLs once all variables found)
- Timeout protection (skip slow/broken URLs)
- Progress tracking per variable, not per URL

Expected time: 15-30 minutes (vs 11+ hours for old version!)

Author: Claude
"""

import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import os
import re
import json
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading

# ============================================================================
# CONFIGURATION
# ============================================================================

WIDE_FILE = "nhanes_wide_aggregated.csv"
DICT_OUTPUT = "nhanes_complete_dictionary.csv"
VALUE_CODES_OUTPUT = "nhanes_value_codes.csv"
CACHE_FILE = "nhanes_scrape_cache.json"
DATA_DIR = "nhanes_data"

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
}

# Optimization settings
MAX_WORKERS = 10  # Process 10 URLs simultaneously
URL_TIMEOUT = 15  # Skip URLs that take longer than 15 seconds
MAX_URLS_TO_CHECK = None  # Check ALL URLs for maximum completeness

# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def load_cache():
    """Load cached scraped data"""
    if os.path.exists(CACHE_FILE):
        try:
            with open(CACHE_FILE, 'r') as f:
                return json.load(f)
        except:
            return {}
    return {}

def save_cache(cache):
    """Save cache to file"""
    with open(CACHE_FILE, 'w') as f:
        json.dump(cache, f, indent=2)

def clean_text(text):
    """Clean up extracted text"""
    if not text:
        return ""
    text = re.sub(r'\s+', ' ', text)
    text = text.strip(' .-:;')
    return text

# ============================================================================
# STREAMLINED EXTRACTION (Faster, focused strategies)
# ============================================================================

def extract_from_tables(soup):
    """Extract from HTML tables"""
    variables = {}

    for table in soup.find_all('table', limit=20):  # Limit to first 20 tables
        rows = table.find_all('tr')
        for row in rows[:100]:  # Limit rows per table
            cells = row.find_all(['td', 'th'])
            if len(cells) >= 2:
                cell_texts = [clean_text(cell.get_text()) for cell in cells]
                potential_var = cell_texts[0]

                if re.match(r'^[A-Z][A-Z0-9_]{1,14}$', potential_var):
                    description = ' '.join(cell_texts[1:])
                    if len(description) > 5:
                        variables[potential_var] = clean_text(description)

    return variables

def extract_from_text_patterns(soup):
    """Extract using text patterns (faster than line-by-line)"""
    variables = {}
    page_text = soup.get_text()

    # Find all patterns like "VARNAME - Description" or "Variable Name: VARNAME"
    patterns = [
        r'([A-Z][A-Z0-9_]{2,14})\s*[-–—:]\s*([^\n]{10,200})',
        r'Variable Name:\s*([A-Z][A-Z0-9_]+)[^\n]*\n[^\n]*(?:English Text|Description|Label):\s*([^\n]+)',
    ]

    for pattern in patterns:
        matches = re.finditer(pattern, page_text, re.IGNORECASE | re.MULTILINE)
        for match in matches:
            var_name = match.group(1).strip()
            description = clean_text(match.group(2) if len(match.groups()) > 1 else match.group(1))
            if len(description) > 5:
                variables[var_name] = description[:500]

    return variables

def extract_value_codes_from_tables(soup):
    """Extract value codes from tables (optimized)"""
    value_codes = {}

    for table in soup.find_all('table', limit=20):
        rows = table.find_all('tr')
        if not rows:
            continue

        header_text = clean_text(rows[0].get_text()).lower()

        # Quick check if this is a code table
        if not any(word in header_text for word in ['code', 'value', 'count']):
            continue

        # Extract codes
        for row in rows[1:50]:  # Limit to first 50 rows
            cells = row.find_all(['td', 'th'])
            if len(cells) >= 2:
                code_text = clean_text(cells[0].get_text())
                desc_text = clean_text(cells[1].get_text())

                try:
                    code = int(float(code_text))
                except:
                    code = code_text

                if code_text and desc_text and 1 < len(desc_text) < 100:
                    skip_words = ['range', 'mean', 'minimum', 'maximum', 'missing', 'refused']
                    if not any(word in desc_text.lower() for word in skip_words):
                        # Store by variable name (extracted from nearby context)
                        value_codes[code] = desc_text

    return value_codes

def extract_value_codes_from_text(soup):
    """Extract value codes from text patterns"""
    value_codes = {}
    page_text = soup.get_text()

    # Pattern: "1 = Male" or "Code 1: Male"
    pattern = r'(?:^|\n)(?:code\s+)?(\d+)\s*[=:]\s*([^\n]{2,80})'
    matches = re.finditer(pattern, page_text, re.IGNORECASE | re.MULTILINE)

    for match in matches:
        try:
            code = int(match.group(1))
            description = clean_text(match.group(2))
            if 1 < len(description) < 100:
                value_codes[code] = description
        except:
            pass

    return value_codes

# ============================================================================
# MAIN SCRAPING FUNCTION (Optimized)
# ============================================================================

def scrape_url_fast(url, needed_variables, cache, lock):
    """
    Scrape a single URL for multiple variables
    OPTIMIZED: Returns as soon as it finds something useful
    """

    # Check if we already have this URL cached
    if url in cache:
        return cache[url]

    try:
        response = requests.get(url, headers=HEADERS, timeout=URL_TIMEOUT)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        results = {}

        # Extract descriptions (fast strategies only)
        var_descriptions = {}
        var_descriptions.update(extract_from_tables(soup))
        var_descriptions.update(extract_from_text_patterns(soup))

        # Extract value codes (for all variables on this page)
        all_value_codes = {}
        all_value_codes.update(extract_value_codes_from_tables(soup))
        all_value_codes.update(extract_value_codes_from_text(soup))

        # Match variables we need
        for var_name in needed_variables:
            if var_name in var_descriptions:
                results[var_name] = {
                    'description': var_descriptions[var_name],
                    'value_codes': dict(all_value_codes),  # Make a copy
                    'source_url': url
                }

        # Cache with lock (thread-safe)
        with lock:
            cache[url] = results

        return results

    except Exception as e:
        # Just skip problematic URLs
        return {}

# ============================================================================
# PARALLEL PROCESSING
# ============================================================================

def scrape_all_parallel(doc_urls, needed_vars, cache):
    """
    Scrape multiple URLs in parallel
    Returns when all variables found OR all URLs checked
    """

    lock = threading.Lock()
    found_variables = {}
    all_value_codes = []

    # Progress tracking
    pbar = tqdm(total=len(needed_vars), desc="Finding variables", unit="var")

    def process_url(url):
        results = scrape_url_fast(url, needed_vars, cache, lock)

        found_count = 0
        with lock:
            for var_name, info in results.items():
                if var_name not in found_variables:
                    found_variables[var_name] = info
                    found_count += 1

                    # Store value codes
                    for code, desc in info.get('value_codes', {}).items():
                        all_value_codes.append({
                            'variable': var_name,
                            'code': code,
                            'description': desc,
                            'source_url': url
                        })

        pbar.update(found_count)

        # Return True if we found everything
        return len(found_variables) >= len(needed_vars)

    # Process URLs in parallel
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = {executor.submit(process_url, url): url for url in doc_urls}

        for future in as_completed(futures):
            try:
                future.result()
            except Exception as e:
                continue

    pbar.close()

    return found_variables, all_value_codes

# ============================================================================
# MAIN EXECUTION
# ============================================================================

print("=" * 70)
print("🚀 CHUNK 9: OPTIMIZED DATA DICTIONARY GENERATOR")
print("=" * 70)
print("⚡ Using parallel processing for 10-20x speedup!")
print()

start_time = time.time()

# Step 1: Load variables
print("Step 1: Loading variable list...")

if not os.path.exists(WIDE_FILE):
    raise FileNotFoundError(f"❌ Wide format file not found: {WIDE_FILE}")

wide_df = pd.read_csv(WIDE_FILE, nrows=0)
needed_vars = set()

for col in wide_df.columns:
    base_var = col.split('_')[0] if '_' in col else col
    needed_vars.add(base_var)

print(f"   → Need definitions for {len(needed_vars)} variables\n")

# Step 2: Find URLs
print("Step 2: Finding documentation URLs...")
doc_urls = set()

if os.path.exists(DATA_DIR):
    for csv_file in os.listdir(DATA_DIR):
        if csv_file.endswith('.csv'):
            try:
                df = pd.read_csv(os.path.join(DATA_DIR, csv_file), nrows=1)
                for col in df.columns:
                    if 'url' in col.lower() or 'doc' in col.lower():
                        if not df[col].isna().all():
                            url = str(df[col].iloc[0])
                            if url.startswith('http'):
                                doc_urls.add(url)
            except:
                continue

doc_urls = sorted(list(doc_urls))
urls_to_check = doc_urls if MAX_URLS_TO_CHECK is None else doc_urls[:MAX_URLS_TO_CHECK]
print(f"   → Found {len(doc_urls)} documentation URLs")
print(f"   → Will check ALL {len(urls_to_check)} URLs for maximum completeness\n")

# Step 3: Load cache
print("Step 3: Loading cache...")
cache = load_cache()
cached_vars = sum(1 for url_results in cache.values() for _ in url_results)
print(f"   → Cache contains {len(cache)} URLs with {cached_vars} variables\n")

# Step 4: Parallel scraping
print("Step 4: Scraping with parallel processing...")
print(f"   → Using {MAX_WORKERS} parallel workers")
print(f"   → Timeout per URL: {URL_TIMEOUT} seconds")
print("-" * 70)

found_variables, all_value_codes = scrape_all_parallel(urls_to_check, needed_vars, cache)

scrape_time = time.time() - start_time

print(f"\n✓ Scraping complete in {scrape_time/60:.1f} minutes!")
print(f"   Variables found: {len(found_variables)}/{len(needed_vars)}")
print(f"   Value codes found: {len(all_value_codes)}\n")

# Save cache
save_cache(cache)
print(f"✓ Cache saved: {CACHE_FILE}\n")

# Step 5: Create dictionary
print("Step 5: Creating data dictionary...")

dict_records = []

for col in wide_df.columns:
    base_var = col.split('_')[0] if '_' in col else col

    if base_var in found_variables:
        info = found_variables[base_var]

        # Handle both old and new cache formats
        if isinstance(info, str):
            # Old cache format - just a description string
            dict_records.append({
                'column_name': col,
                'base_variable': base_var,
                'description': info,
                'value_codes': '',
                'has_codes': 'No',
                'source': 'Scraped (cached)'
            })
            continue

        # Get value codes for this specific variable
        var_codes = [vc for vc in all_value_codes if vc['variable'] == base_var]

        if var_codes:
            # Format as "1=Male; 2=Female"
            unique_codes = {}
            for vc in var_codes:
                if vc['code'] not in unique_codes:
                    unique_codes[vc['code']] = vc['description']

            try:
                sorted_codes=sorted(unique_codes.items())
            except TypeError:
                sorted_codes=sorted(unique_codes.items(), key=lambda x: str(x[0]))

            value_codes_str = '; '.join([
                f"{code}={desc}" for code, desc in sorted_codes
            ])
        else:
            value_codes_str = ""

        dict_records.append({
            'column_name': col,
            'base_variable': base_var,
            'description': info.get('description', f'Variable {base_var}'),
            'value_codes': value_codes_str,
            'has_codes': 'Yes' if value_codes_str else 'No',
            'source': 'Scraped'
        })
    else:
        dict_records.append({
            'column_name': col,
            'base_variable': base_var,
            'description': f'Variable {base_var}',
            'value_codes': '',
            'has_codes': 'Unknown',
            'source': 'Not found'
        })

dict_df = pd.DataFrame(dict_records)
dict_df.to_csv(DICT_OUTPUT, index=False)
print(f"✓ Dictionary saved: {DICT_OUTPUT}\n")

# Step 6: Value codes file
print("Step 6: Creating value codes reference...")

value_codes_df = pd.DataFrame(all_value_codes)
if not value_codes_df.empty:
    # Remove duplicates
    value_codes_df = value_codes_df.drop_duplicates(subset=['variable', 'code'])
    value_codes_df = value_codes_df.sort_values(['variable', 'code'])
    value_codes_df.to_csv(VALUE_CODES_OUTPUT, index=False)
    print(f"✓ Value codes saved: {VALUE_CODES_OUTPUT}\n")
else:
    print("⚠ No value codes found\n")

# Step 7: Statistics
total_time = time.time() - start_time

print("=" * 70)
print("RESULTS SUMMARY")
print("=" * 70)
print(f"⏱️  Total time: {total_time/60:.1f} minutes")
print(f"📊 Total columns: {len(dict_df)}")
print(f"✓  Descriptions found: {len(dict_df[dict_df['source']=='Scraped'])}")
print(f"🏷️  Variables with codes: {len(dict_df[dict_df['has_codes']=='Yes'])}")
print(f"📝 Total value codes: {len(value_codes_df) if not value_codes_df.empty else 0}")
coverage = len(dict_df[dict_df['source']=='Scraped']) / len(dict_df) * 100
print(f"📈 Coverage: {coverage:.1f}%")
print()

# Step 8: Show samples
print("Sample variables with value codes:")
print("-" * 70)
samples = dict_df[dict_df['has_codes'] == 'Yes'].head(5)

if not samples.empty:
    for _, row in samples.iterrows():
        desc = row['description'][:60] + "..." if len(row['description']) > 60 else row['description']
        print(f"\n{row['base_variable']}: {desc}")
        if row['value_codes']:
            codes = row['value_codes'].split('; ')[:3]
            for code in codes:
                print(f"  • {code}")
            remaining = len(row['value_codes'].split('; ')) - 3
            if remaining > 0:
                print(f"  • ... and {remaining} more")

print("\n" + "=" * 70)
print("✅ CHUNK 9 COMPLETE!")
print("=" * 70)
print()
print("💡 Speed improvements:")
print(f"   • Parallel processing: {MAX_WORKERS} workers")
print(f"   • Early termination when all variables found")
print(f"   • Timeout protection for slow URLs")
print(f"   • Smart caching for re-runs")

🚀 CHUNK 9: OPTIMIZED DATA DICTIONARY GENERATOR
⚡ Using parallel processing for 10-20x speedup!

Step 1: Loading variable list...
   → Need definitions for 608 variables

Step 2: Finding documentation URLs...
   → Found 134 documentation URLs
   → Will check ALL 134 URLs for maximum completeness

Step 3: Loading cache...
   → Cache contains 0 URLs with 0 variables

Step 4: Scraping with parallel processing...
   → Using 10 parallel workers
   → Timeout per URL: 15 seconds
----------------------------------------------------------------------


Finding variables:  95%|█████████▍| 577/608 [00:14<00:00, 40.80var/s]



✓ Scraping complete in 0.2 minutes!
   Variables found: 577/608
   Value codes found: 6269

✓ Cache saved: nhanes_scrape_cache.json

Step 5: Creating data dictionary...
✓ Dictionary saved: nhanes_complete_dictionary.csv

Step 6: Creating value codes reference...
✓ Value codes saved: nhanes_value_codes.csv

RESULTS SUMMARY
⏱️  Total time: 0.3 minutes
📊 Total columns: 608
✓  Descriptions found: 577
🏷️  Variables with codes: 495
📝 Total value codes: 6269
📈 Coverage: 94.9%

Sample variables with value codes:
----------------------------------------------------------------------

SDDSRVYR: Data Release Number
  • 0=Never Attended / Kindergarten Only
  • 1=$ 0 to $ 4,999
  • 2=$ 5,000 to $ 9,999
  • ... and 18 more

RIDSTATR: Interview/Examination Status
  • 0=Never Attended / Kindergarten Only
  • 1=$ 0 to $ 4,999
  • 2=$ 5,000 to $ 9,999
  • ... and 18 more

RIAGENDR: Gender
  • 0=Never Attended / Kindergarten Only
  • 1=$ 0 to $ 4,999
  • 2=$ 5,000 to $ 9,999
  • ... and 18 more

RIDAGEY

In [24]:
import os

if os.path.exists("nhanes_scrape_cache.json"):
    os.remove("nhanes_scrape_cache.json")
    print("✓ Deleted old cache")

✓ Deleted old cache
